# 형태소 분석과 표제어 추출

In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
text = "Wikipedia is maintained by volunteers."
doc = nlp(text)

In [4]:
for token in doc:
    print(token.text, 
          token.lemma_,  # 표제어
          token.pos_,    # 단어의 품사(간략)
          token.tag_,    # 자세한 품사(상세)
          token.dep_,    # 문법적 의존 관계
          token.is_stop) # 불용어 여부

Wikipedia Wikipedia PROPN NNP nsubjpass False
is be AUX VBZ auxpass True
maintained maintain VERB VBN ROOT False
by by ADP IN agent True
volunteers volunteer NOUN NNS pobj False
. . PUNCT . punct False


# 용어

In [5]:
spacy.explain('PROPN')

'proper noun'

### 품사: https://universaldependencies.org/u/pos/all.html
### 자세한 영어 품사: https://www.clips.uantwerpen.be/pages/mbsp-tags
### 의존관계: https://universaldependencies.org/u/dep/index.html

# 명사와 동사의 표제어로(특정품사로) 단어 문서 행렬 만들기

In [6]:
words = []

In [7]:
for token in doc:
    if token.tag_[0] == 'N' or token.tag_[0] == 'V':
        words.append(token.lemma_)

In [8]:
words

['Wikipedia', 'be', 'maintain', 'volunteer']

In [9]:
words1 = []
for token in doc:
    if token.tag_[0] in 'NV':
        words1.append(token.lemma_.lower())

In [10]:
words1

['wikipedia', 'be', 'maintain', 'volunteer']

### 명사와 동사의 표제어만 추출하는 함수를 생성

In [11]:
def extract_nv(text):
    doc = nlp(text)
    words = []
    for token in doc:
        if token.tag_[0] in 'NV':
            words.append(token.lemma_.lower()) # 표제어를 소문자로 추출
    return words

In [12]:
extract_nv('Apple is a company')

['apple', 'be', 'company']

### CountVectorizer를 초기화

### 기본적으로 CountVectorizer 빈칸단위로 단어를 끊어서 토근화
### tokenizer=extract_nv 함수를 이용해서 토근을 추출

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500, tokenizer=extract_nv) 

### 데이터를 불러온다.

In [14]:
import pandas as pd
df = pd.read_excel('../data/imdb.xlsx', index_col=0)
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


### TDM을 만든다.

In [15]:
# 동사와 명사의 표제어만 추출하여 빈도를 계산
tdm = cv.fit_transform(df['review']) # 빈칸단위로 끊을 때보다는 시간이 오래 걸림.

### 빈도 순으로 정렬한다.

In [16]:
wc = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '빈도': tdm.sum(axis=0).flat
})

In [17]:
wc.sort_values('빈도', ascending=False).head()

,단어,빈도
49,be,843
281,movie,211
158,film,189
193,have,119
119,do,112
